# All Conference Transfers

In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Build out DataFrames

In [2]:
SQL = '''SELECT * from Transfers247_Unranked'''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
thomasbrown_nebraska_2012,thomasbrown_nebraska,thomasbrown,2012,LB,2
zaireanderson_nebraska_2012,zaireanderson_nebraska,zaireanderson,2012,LB,2
lamarrwoodley_michigan_2003,lamarrwoodley_michigan,lamarrwoodley,2003,LB,2
prescottburgess_michigan_2003,prescottburgess_michigan,prescottburgess,2003,DB,2
shawncrable_michigan_2003,shawncrable_michigan,shawncrable,2003,DE,2
...,...,...,...,...,...
brettmedforth_troy_2013,brettmedforth_troy,brettmedforth,2013,OL,1
cardelllue_troy_2013,cardelllue_troy,cardelllue,2013,DB,2
zachmoore_troy_2013,zachmoore_troy,zachmoore,2013,LB,2


In [3]:
SQL = '''SELECT ID, PlayerName, Year, StandardizedPosition, KeyPositionGroup from UnlinkedAllConference'''
df_ac = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_ac.index.name = 'AllConf_ID'
df_ac

,PlayerName,Year,StandardizedPosition,KeyPositionGroup
AllConf_ID,,,,
joeyashcroft_airforce,joeyashcroft,2002,K,3
anthonyschlegal_airforce,anthonyschlegal,2002,LB,2
wescrawley_airforce,wescrawley,2002,DB,2
marchellograddy_airforce,marchellograddy,2003,LB,2
drewfowler_airforce,drewfowler,2006,LB,2
...,...,...,...,...
dericyaussi_wyoming,dericyaussi,2004,K,3
zachmorris_wyoming,zachmorris,2004,DL,2
jovonbouknight_wyoming,jovonbouknight,2005,WR,1


### NOTES

> You don't have position data, so you are doing the best you can here.  Blocking on exact name match isn't great, but you'll have to change how you collect from wikipedia to change this - cause you'll need to fetch Position data so you can derive position group and block on that.

In [4]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])

In [5]:
pairs = indexer.index(df_ac, df_247)

In [6]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')

In [7]:
features = c.compute(pairs, df_ac, df_247)

In [8]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  \
AllConf_ID         247_IDYR                                     
zachzachpaul_akron caseykreiter_iowa_2009            0.083333   
                   zachhirth_illinois_2012           0.416667   
                   lukenelson_illinois_2012          0.000000   
                   ryanfrain_illinois_2012           0.166667   
                   pjmangieri_nebraska_2009          0.083333   
...                                                       ...   
devinmoore_wyoming nathandouglas_louisiana_2007      0.153846   
                   lancekelley_louisiana_2007        0.090909   
                   jawanzamitchell_louisiana_2007    0.200000   
                   bradmcguire_louisiana_2007        0.181818   
                   julianshankle_louisiana_2007      0.230769   

                                                   StandardizedPosition  \
AllConf_ID         247_IDYR                                               
zachzachpaul_akron caseykreiter_iowa_2009                             0   
                   zachhirth_illinois_2012                            0   
                   lukenelson_illinois_2012                           1   
                   ryanfrain_illinois_2012                            0   
                   pjmangieri_nebraska_2009                           0   
...                                                                 ...   
devinmoore_wyoming nathandouglas_louisiana_2007                       0   
                   lancekelley_louisiana_2007                         0   
                   jawanzamitchell_louisiana_2007                     0   
                   bradmcguire_louisiana_2007                         0   
                   julianshankle_louisiana_2007                       1   

                                                   KeyPositionGroup  Year  \
AllConf_ID         247_IDYR                                                 
zachzachpaul_akron caseykreiter_iowa_2009                         1   1.0   
                   zachhirth_illinois_2012                        1   1.0   
                   lukenelson_illinois_2012                       1   1.0   
                   ryanfrain_illinois_2012                        1   1.0   
                   pjmangieri_nebraska_2009                       1   1.0   
...                                                             ...   ...   
devinmoore_wyoming nathandouglas_louisiana_2007                   1   1.0   
                   lancekelley_louisiana_2007                     1   1.0   
                   jawanzamitchell_louisiana_2007                 1   1.0   
                   bradmcguire_louisiana_2007                     1   1.0   
                   julianshankle_louisiana_2007                   1   1.0   

                                                        Sum  
AllConf_ID         247_IDYR                                  
zachzachpaul_akron caseykreiter_iowa_2009          0.520833  
                   zachhirth_illinois_2012         0.604167  
                   lukenelson_illinois_2012        0.750000  
                   ryanfrain_illinois_2012         0.541667  
                   pjmangieri_nebraska_2009        0.520833  
...                                                     ...  
devinmoore_wyoming nathandouglas_louisiana_2007    0.538462  
                   lancekelley_louisiana_2007      0.522727  
                   jawanzamitchell_louisiana_2007  0.550000  
                   bradmcguire_louisiana_2007      0.545455  
                   julianshankle_louisiana_2007    0.807692  

[3376892 rows x 5 columns]

In [12]:
#features.insert(0, 'sourceID', features.index.get_level_values(0))
#features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    #.929
    if (data['Sum'] > .942):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsAllConferenceTransfer.csv")

In [13]:
dfFinal

,sourceID,targetID,PlayerName,StandardizedPosition,KeyPositionGroup,Year,Sum
"(aaronmerz_california, aaronbern_centralflorida_2003)",aaronmerz_california,aaronbern_centralflorida_2003,0.777778,1,1,1.0,0.944444
"(aaronrogers_california, aaronrodgers_california_2003)",aaronrogers_california,aaronrodgers_california_2003,0.916667,1,1,1.0,0.979167
"(aaronwebster_cincinnati, aaronhester_ucla_2008)",aaronwebster_cincinnati,aaronhester_ucla_2008,0.833333,1,1,1.0,0.958333
"(adamvincent_utep, adamvincent_arizonastate_2004)",adamvincent_utep,adamvincent_arizonastate_2004,1.000000,1,1,1.0,1.000000
"(adrianbushell_louisville, adrianbushell_florida_2008)",adrianbushell_louisville,adrianbushell_florida_2008,1.000000,1,1,1.0,1.000000
...,...,...,...,...,...,...,...
"(tyrellfenroy_louisianalafayette, tyrellfenroy_louisiana_2005)",tyrellfenroy_louisianalafayette,tyrellfenroy_louisiana_2005,1.000000,1,1,1.0,1.000000
"(tyronemckenzie_southflorida, tyronemckenzie_michiganstate_2004)",tyronemckenzie_southflorida,tyronemckenzie_michiganstate_2004,1.000000,1,1,1.0,1.000000
"(valerianezeoke_newmexicostate, valerianumeezeoke_newmexicostate_2011)",valerianezeoke_newmexicostate,valerianumeezeoke_newmexicostate_2011,0.823529,1,1,1.0,0.955882
"(vaneskridge_eastcarolina, vaneskeridge_eastcarolina_2005)",vaneskridge_eastcarolina,vaneskeridge_eastcarolina_2005,0.916667,1,1,1.0,0.979167


In [14]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_ACTransfersUnranked.json",
    fuzzyMI[0:150],
    df_ac,
    df_247,
    dataset_a_name="All Conference",
    dataset_b_name="Master"
)

In [15]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result-acunranked.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [16]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 4, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()